--------------------------------------------------------------------------------
COMUNIDADE DATA SCIENCE - CDS

PROGRAMA DE ENSINO DATA FORMATION

CURSO: FTC - ANALISANDO DADOS COM PYTHON

Projeto do Aluno - DASHBOARD DO CASO 'EMPRESA FOME ZERO'

Aluno: Manoel L. M. Mendonça

Junho-2023

--------------------------------------------------------------------------------

# 0. INITIAL HOUSEKEEPING

## 0.1. Importing Libraries

In [1]:
# STEP-ZERO.zero: IMPORTING LIBRARIES
import pandas as pd
import numpy as np
from haversine import haversine
import plotly.express as px
import plotly.graph_objects as go
import inflection
import folium
from streamlit_folium import folium_static


## 0.2. Code kindly supplied in the assignment

In [2]:
# CODE supplied in the assignment statement
COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zealand",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}

def country_name(country_id):
    return COUNTRIES[country_id]

def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"
    
COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}

def color_name(color_code):
    return COLORS[color_code]


## 0.3. Database Structure - Statistical Table

In [ ]:
# DATABASE COLUMNS:
#   Restaurant ID . . . : numeric (int64). OOPS: Some restaurants are repeated
#   Restaurant Name . . : string. Need: strip
#   Country Code  . . . : numeric (int64), 15 distinct values
#   City  . . . . . . . : string
#   Address . . . . . . : string
#   Locality. . . . . . : string. Need: strip
#   Locality Verbose. . : string. Need: strip
#   Longitude . . . . . : float
#   Latitude  . . . . . : float
#   Cuisines  . . . . . : string, with nan occurrences
#   Average Cost for two: int. Question: in which currency??
#   Currency  . . . . . : string
#   Has Table booking   : int, values = [ 0, 1 ]
#   Has Online delivery : int, values = [ 0, 1 ]
#   Is delivering now   : int, values = [ 0, 1 ]
#   Switch to order menu: int, value = [ 0 ]  -->> No need. Exclude !!
#   Price range . . . . : int, values = [ 1, 2, 3, 4 ]
#   Aggregate rating. . : float, values = [ 0. , ..., 4.9 ]
#   Rating color. . . . : string. Ex: '9ACD32', 'CBCBC8'
#   Rating text . . . . : string.
#   Votes . . . . . . . : int. From 0 to 41333.

## 0.4. Database Cleansing Class

In [3]:
#-------------------------------------------------------------------------------
#
# DATABASE CLEANSING CLASS
#                                                           mlmm in 19-june-2023
#-------------------------------------------------------------------------------

class DataBaseCleansing():

    #---------------------------------------------------------------------------
    # CONSTRUCTOR
    #---------------------------------------------------------------------------
    def __init__(self) -> None:
        self.dtframe = None
        return
    
    def __init__(self, inDataframe) -> None:
        self.dtframe = inDataframe.copy()
        return
    
    #---------------------------------------------------------------------------
    def LoadDataframe(self, inDataframe) -> None:
        self.dtframe = inDataframe.copy()
        return

    #---------------------------------------------------------------------------
    # Perform the main general cleansing operations
    #---------------------------------------------------------------------------
    def GeneralCleansing(self) -> pd.core.frame.DataFrame:

        # First op: rename columns
        self.rename_columns()

        # Remove duplicated restaurant regs
        self.dtframe.drop_duplicates(subset=['restaurant_id'], inplace=True)

        # Remove 'switch_to_order_menu' column: not needed
        self.dtframe.drop( columns=['switch_to_order_menu'] )

        # Eliminate white spaces at beginning & ending
        self.StripColumns()

        # Create 'UniqueCuisine' column
        self.CreateUniqueCuisine()

        return self.dtframe

    #---------------------------------------------------------------------------
    # Adjust column names, extract white spaces, etc
    #---------------------------------------------------------------------------
    def rename_columns( self ) -> pd.core.frame.DataFrame:
    
        # Kindly offered by the teacher :)
        title = lambda x: inflection.titleize(x)
        snakecase = lambda x: inflection.underscore(x)
        spaces = lambda x: x.replace(" ", "")
        cols_old = list(self.dtframe.columns)
        cols_old = list(map(title, cols_old))
        cols_old = list(map(spaces, cols_old))
        cols_new = list(map(snakecase, cols_old))
        self.dtframe.columns = cols_new

        # create 'country_name' column
        self.dtframe['country_name'] = ( self.dtframe.loc[:, 'country_code']
                                             .apply(lambda x: country_name(x)) )

        return self.dtframe

    #---------------------------------------------------------------------------
    # Apply strip() command to some columns.
    #---------------------------------------------------------------------------
    def StripColumns(self) -> None:
        self.dtframe['restaurant_name'] = self.dtframe['restaurant_name'].str.strip()
        self.dtframe['city'] = self.dtframe['city'].str.strip()
        self.dtframe['locality'] = self.dtframe['locality'].str.strip()
        self.dtframe['locality_verbose'] = self.dtframe['locality_verbose'].str.strip()
        return

    #---------------------------------------------------------------------------
    # Create column to receive the first type of Cuisine shown in 'Cuisines' column
    #---------------------------------------------------------------------------
    #
    # When running command [ dbc.dtframe.isna().sum() ] to discover number of NAN's...
    # ...we noticed that 'Cuisines' column has 13. Other columns do not have it.
    # Solution: when 'Cuisines' = nan, make 'UniqueCuisine' = ""
    #
    def CreateUniqueCuisine(self) -> None:
        self.dtframe['unique_cuisine'] = ( self.dtframe.loc[:, 'cuisines']
                                  .apply(lambda x: x.split(",")[0] if isinstance(x, str) else "") )
        return


## 0.5. Import Database

In [4]:
#
# STEP-ZERO.one: IMPORTING DATABASE
#
# Previous internal company dashboard:
#     https://pedroferraresi-ftc-analise-python-01--main-page-ozp460.streamlit.app/
# Database Source:
#     https://www.kaggle.com/datasets/akashram/zomato-restaurants-autoupdated-dataset?resource=download&select=zomato.csv
# Read CSV data at:
#     http://menezes.mendonca.nom.br/datasets/zomato.csv
#
df = pd.read_csv( 'http://menezes.mendonca.nom.br/datasets/zomato.csv' )
df.head(2)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619


## 0.6. Clean Data

In [5]:
#
# STEP-ZERO.two: VERIFYING & CLEANSING DATABASE
#
dbc = DataBaseCleansing(df)
df1 = dbc.GeneralCleansing()
df1.head(2)

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,country_name,unique_cuisine
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,0,3,4.6,3F7E00,Excellent,619,Philippines,Italian
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",...,0,0,0,4,4.7,3F7E00,Excellent,469,Philippines,European


# 1. VISÃO GERAL (ok)

## 1.1. Quantos restaurantes únicos estão registrados?

In [10]:
print(f"Quantidade de restaurantes únicos = {len( df1['restaurant_name'].unique() )}")

Quantidade de restaurantes únicos = 5904


## 1.2. Quantos países únicos estão registrados?

In [11]:
print(f"Quantidade de países = {len( df1['country_code'].unique() )}")

Quantidade de países = 15


In [19]:
# lista de nomes de países na base de dados
df2 = df1['country_name'].unique().tolist()
df3 = sorted(df2)
df3

['Australia',
 'Brazil',
 'Canada',
 'England',
 'India',
 'Indonesia',
 'New Zealand',
 'Philippines',
 'Qatar',
 'Singapure',
 'South Africa',
 'Sri Lanka',
 'Turkey',
 'United Arab Emirates',
 'United States of America']

## 1.3. Quantas cidades únicas estão registradas?

In [13]:
print(f"Quantidade de cidades = {len( df1['city'].unique() )}")

Quantidade de cidades = 125


## 1.4. Qual o total de avaliações feitas?

In [24]:
# Move 'votes' to a series and sum it up
df2 = df1['votes']
Total_of_evaluations = df2.sum()
Total_of_evaluations

4195634

## 1.5. Qual o total de tipos de culinária registrados?

In [17]:
print(f"Quantidade de culinárias = {len( df1['unique_cuisine'].unique() )}")

# OBSERVAÇAO: fizemos usando apenas a primeira culinária de cada restaurante.
#             Mas pode haver alguma culinária nova escondida da 2ª prá traz...

Quantidade de culinárias = 166


## extra 1.6. Informações para montagem do MAPA MUNDI

In [18]:
# Refazendo o Global Map
colunas = ['city','aggregate_rating','latitude','longitude']
df2 = df1.loc[: ,colunas]
df3 = ( df2.loc[:,:].groupby(['city','aggregate_rating'])
                    .median()
                    .reset_index() )
# Desenhar MAPA
CityMap = folium.Map( zoom_start=11 )

for index, location_info in df3.iterrows():
    # Insere, um por um, os pinos no mapa.
    folium.Marker( [location_info['latitude'],
                    location_info['longitude']],
                    popup=location_info[['city','aggregate_rating']] ).add_to(CityMap)
folium_static( CityMap, width=1024, height=600 )

df1.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'country_name', 'unique_cuisine'],
      dtype='object')

In [19]:
df1['rating_color'].unique()

array(['3F7E00', '5BA829', '9ACD32', 'CDD614', 'FFBA00', 'CBCBC8',
       'FF7800'], dtype=object)

# 2. VISÃO PAÍS (ok)

## 2.1. Qual o nome do país que possui mais cidades registradas?

In [9]:
colunas = ['country_name','city']
df2 = ( df1.loc[:, colunas]
           .groupby('country_name').count()
           .sort_values(by=['city'], ascending=False)
           .reset_index() )

Country_with_More_Cities = df2.loc[0, 'country_name']

print(f"O país com mais cidades é: {Country_with_More_Cities}")
df2

O país com mais cidades é: India


,country_name,city
0,India,3120
1,United States of America,1378
2,England,400
3,South Africa,346
4,United Arab Emirates,300
5,Brazil,240
6,New Zealand,239
7,Australia,180
8,Canada,180
9,Turkey,159


In [11]:
# Listar os primeiros 6 países com mais restaurantes?
colunas = ['country_name','restaurant_id']
df2 = ( df1.loc[:, colunas]
           .groupby('country_name').count()
           .sort_values(by=['restaurant_id'], ascending=False)
           .reset_index() )

Country_with_More_Rests = df2.loc[0:5, 'country_name']
df3 = Country_with_More_Rests.tolist()
df3

['India',
 'United States of America',
 'England',
 'South Africa',
 'United Arab Emirates',
 'Brazil']

In [22]:
# OBTER TODOS OS REGISTROS QUE SE REFIRAM A ALGUM PAÍS DA LISTA DE ENTRADA (df3):

# pega a lista, df3, faz o "any" e coloca isso dentro do "lambda". Vejamos:
linhas2 = ( df1.loc[:, 'country_name'].apply( lambda x: any(pais in x for pais in df3) ) )
df5 = df1.loc[linhas2, colunas].copy()
# parece que funcionou
df6 = df5['country_name'].unique()
df6
type(df1)

pandas.core.frame.DataFrame

## 2.2. Qual o nome do país que possui mais restaurantes registrados?

In [7]:
colunas = ['country_name','restaurant_id']
df2 = ( df1.loc[:, colunas]
           .groupby('country_name').count()
           .sort_values(by=['restaurant_id'], ascending=False)
           .reset_index() )
Country_with_More_Restaurants = df2.loc[0, 'country_name']

print(f"O país com mais restaurantes é: {Country_with_More_Restaurants}")
df2

O país com mais restaurantes é: India


,country_name,restaurant_id
0,India,3120
1,United States of America,1378
2,England,400
3,South Africa,346
4,United Arab Emirates,300
5,Brazil,240
6,New Zealand,239
7,Australia,180
8,Canada,180
9,Turkey,159


## 2.3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [107]:
colunas = ['country_name','restaurant_id','price_range']
#   Price range . . . . : int, values = [ 1, 2, 3, 4 ]
df2 = df1.loc[ df1['price_range']==4 , colunas].copy()
df3 = ( df2.loc[:, ['country_name','restaurant_id']]
           .groupby('country_name').count()
           .sort_values(by=['restaurant_id'] , ascending=False )
           .reset_index() )
Country_with_More_Level4 = df3.loc[0, 'country_name']

print(f"O país com mais restaurantes nível 4 é: {Country_with_More_Level4}")
df3

O país com mais restaurantes nível 4 é: United States of America


## 2.4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [109]:
colunas = ['country_name','unique_cuisine','restaurant_id']
linhas = df1['unique_cuisine'] != ""
df2 = ( df1.loc[linhas, colunas]
           .groupby(['country_name','unique_cuisine'])
           .count().reset_index() )
df3 = ( df2.loc[:, ['country_name','unique_cuisine']]
           .groupby('country_name').count()
           .sort_values(by=['unique_cuisine'] , ascending=False )
           .reset_index() )
Country_with_More_Cuisines = df3.loc[0, 'country_name']

print(f"O país com maior quantidade de tipos culinários é: {Country_with_More_Cuisines}")
df3

O país com maior quantidade de tipos culinários é: India


## 2.5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [10]:
colunas = ['country_name','votes']
df2 = ( df1.loc[:, colunas]
           .groupby('country_name').sum()
           .sort_values(by=['votes'] , ascending=False )
           .reset_index() )
Country_with_More_Evaluations = df2.loc[0, 'country_name']

print(f"O país com maior quantidade de avaliações é: {Country_with_More_Evaluations}")
df2

O país com maior quantidade de avaliações é: India


,country_name,votes
0,India,2800199
1,United States of America,523414
2,United Arab Emirates,177564
3,Australia,130357
4,Canada,105018
5,Turkey,100193
6,Indonesia,89026
7,South Africa,81939
8,New Zealand,52532
9,Philippines,48398


## 2.6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [113]:
#   Has Online delivery : int, values = [ 0, 1 ]
colunas = ['country_name','has_online_delivery','restaurant_id']
linhas = df1['has_online_delivery'] == 1
df2 = df1.loc[linhas, colunas]
colunas = ['country_name','restaurant_id']
df3 = ( df2.loc[:, colunas]
           .groupby('country_name').count()
           .sort_values(by=['restaurant_id'] , ascending=False )
           .reset_index() )
Country_with_More_Deliveries = df3.loc[0, 'country_name']

print(f"O país com maior quantidade de restaurantes que fazem entregas é: {Country_with_More_Evaluations}")
df3

O país com maior quantidade de restaurantes que fazem entregas é: India


## 2.7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [120]:
#   Has Table booking   : int, values = [ 0, 1 ]
colunas = ['country_name','restaurant_id','has_table_booking']
linhas = df1['has_table_booking'] == 1
df2 = df1.loc[linhas, colunas]
colunas = ['country_name','restaurant_id']
df3 = ( df2.loc[:, colunas]
           .groupby('country_name').count()
           .sort_values(by=['restaurant_id'] , ascending=False )
           .reset_index() )
Country_with_More_BookingTables = df3.loc[0, 'country_name']

print(f"O país com maior quantidade de restaurantes que aceitam reservas é: {Country_with_More_BookingTables}")
df3

O país com maior quantidade de restaurantes que aceitam reservas é: India


## 2.8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [8]:
# Qual média? Resposta: entendo que seja a média = (total de avaliações do país)/(total de restaurantes do país)
# Numerador: total de avaliações de cada país
colunas = ['country_code','country_name','votes']
NUM = ( df1.loc[:, colunas]
           .groupby(['country_code','country_name']).sum()
           .sort_values(by=['country_code'] )
           .reset_index() )
# Denominador: total de restaurantes do país
colunas = ['country_code','country_name','restaurant_id']
DEN = ( df1.loc[:, colunas]
           .groupby(['country_code','country_name']).count()
           .sort_values(by=['country_code'] )
           .reset_index() )
# Junta colunas de votos e de restaurantes
NUM = NUM.join( DEN['restaurant_id'] )
# Calcula a média por país:
NUM['mean_qty_votes_by_country'] = NUM['votes'] / NUM['restaurant_id']
NUM = NUM.loc[:, :].sort_values(by=['mean_qty_votes_by_country'], ascending=False).reset_index()
country_with_biggest_mean = NUM.loc[0, 'country_name']
biggest_mean = NUM.loc[0, 'mean_qty_votes_by_country']
print(f"'{country_with_biggest_mean}' é o país com maior quantidade média de avaliações, com {round(biggest_mean,0)} votos por restaurante")
#display(NUM)

'Indonesia' é o país com maior quantidade média de avaliações, com 1113.0 votos por restaurante


#### OBS: consultei a monitoria em 22-6-2023, que me sugeriu a seguinte solução melhor:

In [10]:
# SUGESTÃO DA MONITORIA EM 22-JUN-2023

# Numerador: total de avaliações de cada país
colunas = ['country_code','country_name','votes']
NUM = ( df1.loc[:, colunas]
           .groupby(['country_code','country_name']).mean()  # troca de 'sum' por 'mean'
           .sort_values(by=['votes'], ascending=False )      # troquei para 'votes'
           .reset_index() )
# a resposta será o país que aparece no início da lista...
country_with_biggest_mean = NUM.loc[0, 'country_name']
biggest_mean = NUM.loc[0, 'votes']
print(f"'{country_with_biggest_mean}' é o país com maior quantidade média de avaliações, com {round(biggest_mean,0)} votos por restaurante")
# FIM !

'Indonesia' é o país com maior quantidade média de avaliações, com 1113.0 votos por restaurante


## 2.9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [23]:
# Qual média? 
# Resposta: cada restaurante tem sua nota agregada (aggregate_rating)...
#           ...que pode ser sopesada pela quantidade de votos (votes)
#           Assim, a média do país = sum( votes*aggregate_rating ) / sum( votes )

# Dado auxiliar ao Numerador: votes*aggregate_rating
colunas = ['restaurant_id','country_code','country_name','votes','aggregate_rating']
multpl = df1.loc[:, colunas].copy()
multpl['votes_x_rating'] = multpl['votes'] * multpl['aggregate_rating']

# Numerador: sum( votes*aggregate_rating )
# Denominador: sum( votes )
colunas = ['country_code','country_name','votes','votes_x_rating']
NUM_DENOM = ( multpl.loc[:, colunas]
                    .groupby(['country_code','country_name']).sum()
                    .sort_values(by=['country_code'])
                    .reset_index() )
NUM_DENOM

# Calcula média de notas e faz ranking de países
NUM_DENOM['mean_country_rating'] = NUM_DENOM['votes_x_rating'] / NUM_DENOM['votes']
NUM_DENOM = NUM_DENOM.sort_values(by=['mean_country_rating'], ascending=False).reset_index()

country_max_mean_rating = NUM_DENOM.loc[0, 'country_name']
biggest_max_mean_rating = NUM_DENOM.loc[0, 'mean_country_rating']

print(f"'{country_max_mean_rating}' é o país com maior nota média de avaliações, com {round(biggest_max_mean_rating,2)}")
#display(NUM_DENOM)

'Indonesia' é o país com maior nota média de avaliações, com 4.67


#### Solução mais simples: fazer média simples dos ratings, sem se importar com qual restaurante teve mais votos

In [21]:
# Qual média simples? 
# Resposta: seria a média da nota agregada (aggregate_rating)...
#           ...sem considerar a quantidade de votos (votes)
colunas = ['country_code','country_name','aggregate_rating']
df2 = ( df1.loc[:, colunas]
           .groupby(['country_code','country_name']).mean()
           .sort_values('aggregate_rating', ascending=False)
           .reset_index() )
df2
# OBS: prefiro a primeira solução, por ser mais correta.

,country_code,country_name,aggregate_rating
0,94,Indonesia,4.601250
1,162,Philippines,4.463750
2,184,Singapure,4.435000
3,216,United States of America,4.402322
4,14,Australia,4.372778
5,37,Canada,4.321667
6,208,Turkey,4.310063
7,166,Qatar,4.240000
8,148,New Zealand,4.164435
9,189,South Africa,4.061561


## 2.10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [12]:
# Segue a lógica do tópico 2.9:
# Dado auxiliar ao Numerador: votes*aggregate_rating
colunas = ['restaurant_id','country_code','country_name','votes','aggregate_rating']
multpl = df1.loc[:, colunas].copy()
multpl['votes_x_rating'] = multpl['votes'] * multpl['aggregate_rating']

# Numerador: sum( votes*aggregate_rating )
# Denominador: sum( votes )
colunas = ['country_code','country_name','votes','votes_x_rating']
NUM_DENOM = ( multpl.loc[:, colunas]
                    .groupby(['country_code','country_name']).sum()
                    .sort_values(by=['country_code'])
                    .reset_index() )

# Calcula média de notas e faz ranking de países, do menor para o maior
NUM_DENOM['mean_country_rating'] = NUM_DENOM['votes_x_rating'] / NUM_DENOM['votes']
NUM_DENOM = NUM_DENOM.sort_values(by=['mean_country_rating']).reset_index()

country_min_mean_rating = NUM_DENOM.loc[0, 'country_name']
biggest_min_mean_rating = NUM_DENOM.loc[0, 'mean_country_rating']

print(f"'{country_min_mean_rating}' é o país com menor nota média de avaliações, com {round(biggest_min_mean_rating,3)}")
display(NUM_DENOM)

'Sri Lanka' é o país com menor nota média de avaliações, com 4.1


,index,country_code,country_name,votes,votes_x_rating,mean_country_rating
0,10,191,Sri Lanka,12133,49741.8,4.099712
1,2,30,Brazil,2977,12520.8,4.205845
2,9,189,South Africa,81939,348375.8,4.251648
3,7,166,Qatar,30106,128429.9,4.265924
4,12,214,United Arab Emirates,177564,760488.4,4.282897
5,0,1,India,2800199,12091975.6,4.318256
6,5,148,New Zealand,52532,227398.9,4.328769
7,3,37,Canada,105018,459239.4,4.372959
8,11,208,Turkey,100193,440173.5,4.393256
9,1,14,Australia,130357,573965.4,4.403027


## 2.11. Qual a média de preço de um prato para dois por país?

In [13]:
colunas = ['country_code','country_name','currency','average_cost_for_two']
df2 = ( df1.loc[:, colunas]
           .groupby(['country_code','country_name','currency']).mean()
           .sort_values('average_cost_for_two', ascending=False)
           .reset_index() )
df2

,country_code,country_name,currency,average_cost_for_two
0,94,Indonesia,Indonesian Rupiah(IDR),303000.000000
1,14,Australia,Dollar($),138959.783333
2,191,Sri Lanka,Sri Lankan Rupee(LKR),2579.375000
3,162,Philippines,Botswana Pula(P),1227.825000
4,1,India,Indian Rupees(Rs.),703.602564
5,189,South Africa,Rand(R),339.228324
6,166,Qatar,Qatari Rial(QR),174.000000
7,214,United Arab Emirates,Emirati Diram(AED),153.716667
8,184,Singapure,Dollar($),141.437500
9,30,Brazil,Brazilian Real(R$),138.812500


# 3. VISÃO CIDADE (ok)

## 3.1. Qual o nome da cidade que possui mais restaurantes registrados?

In [24]:
colunas = ['country_name','city','restaurant_id']
df2 = ( df1.loc[:, colunas]
           .groupby(['country_name','city']).count()
           .sort_values(by='restaurant_id', ascending=False)
           .sort_values(by=['country_name','city'])
           .reset_index() )
max_restaurants = df2.loc[0, 'restaurant_id']
linhas = df2['restaurant_id'] == 80
df3 = df2.loc[linhas, ['country_name','city']].reset_index()
print("As cidades (empatadas) com a maior quantidade de restaurantes registrados são:")
df3

As cidades (empatadas) com a maior quantidade de restaurantes registrados são:


,index,country_name,city
0,3,Brazil,Brasília
1,4,Brazil,Rio de Janeiro
2,5,Brazil,São Paulo
3,9,England,Birmingham
4,10,England,Edinburgh
5,11,England,Glasgow
6,12,England,London
7,13,England,Manchester
8,14,India,Agra
9,16,India,Allahabad


## 3.2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [30]:
#   Aggregate rating. . : float, values = [ 0. , ..., 4.9 ]
colunas = ['country_name','city','aggregate_rating']
linhas = df1['aggregate_rating'] > 4.0
df2 = df1.loc[linhas, colunas].copy()
df3 = ( df2.loc[:, colunas]
           .groupby(['country_name','city']).count()
           .sort_values(by='aggregate_rating', ascending=False)
           .reset_index() )
# Colecting answer
city_level4 = df3.loc[0,'city']
country_level4 = df3.loc[0,'country_name']
print(f"A cidade com mais restaurantes ranqueados acima de 4 é: {city_level4} em {country_level4}")

A cidade com mais restaurantes ranqueados acima de 4 é: Bangalore em India


## 3.3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [29]:
#   Aggregate rating. . : float, values = [ 0. , ..., 4.9 ]
colunas = ['country_name','city','aggregate_rating']
linhas = df1['aggregate_rating'] < 2.5
df2 = df1.loc[linhas, colunas].copy()
df3 = ( df2.loc[:, colunas]
           .groupby(['country_name','city']).count()
           .sort_values(by='aggregate_rating', ascending=False)
           .reset_index() )
# Colecting answer
city_level25 = df3.loc[0,'city']
country_level25 = df3.loc[0,'country_name']
print(f"A cidade com mais restaurantes ranqueados abaixo de 2,5 é: {city_level25} em {country_level25}")

A cidade com mais restaurantes ranqueados abaixo de 2,5 é: Gangtok em India


## 3.4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [17]:
#   Average Cost for two: int. Question: in which currency??
colunas = ['country_name','city','currency','average_cost_for_two']
df2 = ( df1.loc[:, colunas]
           .groupby(['country_name','city','currency']).mean()
           .sort_values(by='average_cost_for_two', ascending=False)
           .reset_index() )

# Colecting answer
city_costfor2 = df2.loc[0,'city']
country_costfor2 = df2.loc[0,'country_name']
mean_costfor2 = round( df2.loc[0,'average_cost_for_two'],2 )
currency_costfor2 = df2.loc[0,'currency']
print("A cidade com o maior valor médio de um prato para dois é:")
print(f"{city_costfor2} em {country_costfor2}")
print(f"Custo médio = {mean_costfor2} na moeda local '{currency_costfor2}'")

A cidade com o maior valor médio de um prato para dois é:
Adelaide em Australia
Custo médio = 416734.13 na moeda local 'Dollar($)'


## 3.5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [71]:
# Monta df com as culinarias únicas por cidade
colunas = ['country_name','city','unique_cuisine','restaurant_id']
df2 = df1.loc[:, colunas].groupby(['country_name','city','unique_cuisine']).count().reset_index()
# Conta quantas culinárias únicas por cidade
colunas = ['country_name','city','unique_cuisine']
df3 = df2.loc[:, colunas].groupby(['country_name','city']).count().sort_values(by='unique_cuisine', ascending=False).reset_index()
#display(df3)
# Coleta os resultados
city_max_cuisines = df3.loc[0, 'city']
country_max_cuisines = df3.loc[0, 'country_name']
print(f"A cidade com a maior quantidade de tipos culinários é: {city_max_cuisines} em {country_max_cuisines}")

A cidade com a maior quantidade de tipos culinários é: Birmingham em England


## 3.6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [86]:
#  *Has Table booking   : int, values = [ 0, 1 ]
#   Has Online delivery : int, values = [ 0, 1 ]
#   Is delivering now   : int, values = [ 0, 1 ]

colunas = ['country_name','city','has_table_booking']
linhas = (df1['has_table_booking'] == 1)
df2 = df1.loc[linhas, colunas].groupby(['country_name','city']).count().sort_values(by='has_table_booking', ascending=False).reset_index()
# Coleta resultados
city_max_bookings = df2.loc[0, 'city']
country_city_max_bookings = df2.loc[0, 'country_name']
print(f"A cidade com a maior quantidade de restaurantes que fazem reservas é: {city_max_bookings} em {country_city_max_bookings}")

A cidade com a maior quantidade de restaurantes que fazem reservas é: Bangalore em India


## 3.7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [89]:
#   Has Table booking   : int, values = [ 0, 1 ]
#  *Has Online delivery : int, values = [ 0, 1 ]
#   Is delivering now   : int, values = [ 0, 1 ]

colunas = ['country_name','city','has_online_delivery']
linhas = (df1['has_online_delivery'] == 1)
df2 = df1.loc[linhas, colunas].groupby(['country_name','city']).count().sort_values(by='has_online_delivery', ascending=False).reset_index()
# Coleta resultados
city_max_deliveries = df2.loc[0, 'city']
country_city_max_deliveries = df2.loc[0, 'country_name']
print(f"A cidade com a maior quantidade de restaurantes que fazem reservas é: {city_max_deliveries} em {country_city_max_deliveries}")

A cidade com a maior quantidade de restaurantes que fazem reservas é: Bhopal em India


## 3.8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [92]:
#   Has Table booking   : int, values = [ 0, 1 ]
#   Has Online delivery : int, values = [ 0, 1 ]
#  *Is delivering now   : int, values = [ 0, 1 ]

colunas = ['country_name','city','is_delivering_now']
linhas = (df1['is_delivering_now'] == 1)
df2 = df1.loc[linhas, colunas].groupby(['country_name','city']).count().sort_values(by='is_delivering_now', ascending=False).reset_index()
# Coleta resultados
city_max_online = df2.loc[0, 'city']
country_city_max_online = df2.loc[0, 'country_name']
print(f"A cidade com a maior quantidade de restaurantes que fazem reservas é: {city_max_online} em {country_city_max_online}")

A cidade com a maior quantidade de restaurantes que fazem reservas é: Amritsar em India


# 4. VISÃO RESTAURANTES (ok)

## 4.1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [124]:
colunas = ['restaurant_id','restaurant_name','votes']
df2 = df1.loc[:, colunas].sort_values(by='votes',ascending=False).reset_index()
restaurant_most_voted = df2.loc[0, 'restaurant_name']
print(f"O restaurante com maior quantidade de avaliações é: {restaurant_most_voted}")

O restaurante com maior quantidade de avaliações é: Bawarchi


## 4.2. Qual o nome do restaurante com a maior nota média?

In [103]:
colunas = ['restaurant_id','restaurant_name','aggregate_rating']
df2 = df1.loc[:, colunas].sort_values(by='aggregate_rating',ascending=False).reset_index()
restaurant_hihger_rating = df2.loc[0, 'aggregate_rating']
linhas = df2['aggregate_rating']==restaurant_hihger_rating
df3 = df2.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
restaurant_hihger_rating_name = df3.loc[0, 'restaurant_name']
# Show results
print(f"A maior nota dada a um restaurante é: {restaurant_hihger_rating}")
print(f"O restaurante com a maior nota é: {restaurant_hihger_rating_name}")

A maior nota dada a um restaurante é: 4.9
O restaurante com a maior nota é: Indian Grill Room


## 4.3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [107]:
colunas = ['restaurant_id','restaurant_name','average_cost_for_two']
df2 = df1.loc[:, colunas].sort_values(by='average_cost_for_two',ascending=False).reset_index()
restaurant_hihger_cost2 = df2.loc[0, 'average_cost_for_two']
linhas = df2['average_cost_for_two']==restaurant_hihger_cost2
df3 = df2.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
restaurant_hihger_cost2_name = df3.loc[0, 'restaurant_name']
# Show results
print(f"O maior preço de prato para dois é: {restaurant_hihger_cost2}")
print(f"O restaurante esse maior preço é: {restaurant_hihger_cost2_name}")

O maior preço de prato para dois é: 25000017
O restaurante esse maior preço é: d'Arry's Verandah Restaurant


## 4.4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [117]:
colunas = ['restaurant_id','restaurant_name','aggregate_rating']
linhas = df1['unique_cuisine']=='Brazilian'
df2 = df1.loc[linhas, colunas].sort_values(by=['aggregate_rating','restaurant_id']).reset_index()
# Coleta resultados
br_restaurant_lowest_rating = df2.loc[0, 'aggregate_rating']
br_restaurant_lowest_rating_name = df2.loc[0, 'restaurant_name']
print(f"A menor avaliação para restaurante de culinária brasileira é = {br_restaurant_lowest_rating}")
print(f"O restaurante de culinária de tipo brasileira com menor avaliação é: {br_restaurant_lowest_rating_name}")

A menor avaliação para restaurante de culinária brasileira é = 0.0
O restaurante de culinária de tipo brasileira com menor avaliação é: Loca Como tu Madre


## 4.5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [122]:
colunas = ['restaurant_id','restaurant_name','aggregate_rating']
linhas = (df1['unique_cuisine']=='Brazilian') & (df1['country_name']=='Brazil')
df2 = df1.loc[linhas, colunas].sort_values(by='aggregate_rating', ascending=False).reset_index()
higher_rating = df2.loc[0, 'aggregate_rating']
linhas = df2['aggregate_rating']==higher_rating
df3 = df2.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
br_restaurant_higher_rating_name = df3.loc[0, 'restaurant_name']
print(f"No Brasil, o restaurante de culinária brasileira com maior avaliação é: {br_restaurant_higher_rating_name}")

No Brasil, o restaurante de culinária brasileira com maior avaliação é: Braseiro da Gávea


## 4.6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [128]:
#   Has Table booking   : int, values = [ 0, 1 ]
#   Has Online delivery : int, values = [ 0, 1 ]
#  *Is delivering now   : int, values = [ 0, 1 ]
colunas = ['is_delivering_now','votes']
df2 = ( df1.loc[:, colunas]
           .groupby('is_delivering_now').mean()
           .sort_values(by='is_delivering_now')
           .reset_index() )
not_delivering = df2.loc[0,'votes']
delivering = df2.loc[1,'votes']
delivering_has_more_votes = True
if delivering > not_delivering:
    print("SIM. Os que aceitam pedidos online são os mais avaliados")
else:
    delivering_has_more_votes = False
    print("NÃO. Os mais avaliados são os que não aceitam pedidos online")
display(df2)

SIM. Os que aceitam pedidos online são os mais avaliados


,is_delivering_now,votes
0,0,575.861955
1,1,739.228548


## 4.7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [130]:
#   Has Table booking   : int, values = [ 0, 1 ]
#   Has Online delivery : int, values = [ 0, 1 ]
#  *Is delivering now   : int, values = [ 0, 1 ]
colunas = ['is_delivering_now','average_cost_for_two']
df2 = ( df1.loc[:, colunas]
           .groupby('is_delivering_now').mean()
           .sort_values(by='is_delivering_now')
           .reset_index() )
not_delivering = df2.loc[0,'average_cost_for_two']
delivering = df2.loc[1,'average_cost_for_two']
delivering_cost_more_for2 = True
if delivering > not_delivering:
    print("SIM. Os que aceitam pedidos online são os que têm maior valor médio de prato para dois")
else:
    delivering_cost_more_for2 = False
    print("NÃO. Os mais caros para duas pessoas são os que não aceitam pedidos online")
display(df2)

NÃO. Os mais caros para duas pessoas são os que não aceitam pedidos online


,is_delivering_now,average_cost_for_two
0,0,8979.354276
1,1,543.704620


## 4.8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [149]:
colunas = ['country_code','country_name','unique_cuisine','average_cost_for_two']
linhas = (df1['country_name']=='United States of America') & ((df1['unique_cuisine']=='Japanese') | (df1['unique_cuisine']=='BBQ'))
df2 = df1.loc[linhas, colunas].groupby(['country_code','country_name','unique_cuisine']).mean().sort_values(by='unique_cuisine').reset_index()
us_bbq_for2 = df2.loc[0, 'average_cost_for_two']
us_jap_for2 = df2.loc[1, 'average_cost_for_two']
us_jap_more_expensive_than_bbq = False
if us_bbq_for2 > us_jap_for2:
    print("NÃO. Nos EUA, o churrasco é mais caro do que a culinária japonesa")
else:
    us_jap_more_expensive_than_bbq = True
    print("SIM. Nos EUA, culinária japonesa é mais cara do que churrasco")
df2

SIM. Nos EUA, culinária japonesa é mais cara do que churrasco


,country_code,country_name,unique_cuisine,average_cost_for_two
0,216,United States of America,BBQ,39.642857
1,216,United States of America,Japanese,56.406250


# 5. VISÃO CULINÁRIA (ok)

## 5.1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [6]:
colunas = ['restaurant_id','restaurant_name','unique_cuisine','aggregate_rating']
linhas = (df1['unique_cuisine']=='Italian')
df2 = df1.loc[linhas, colunas].copy()
df3 = df2.sort_values(by='aggregate_rating', ascending=False).reset_index()
max_rating = df3.loc[0, 'aggregate_rating']
linhas = df3['aggregate_rating']==max_rating
df4 = df3.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
italian_food_best_ranked = df4.loc[0,'restaurant_name']
print(f"O restaurante de culinária italiana melhor avaliado é: {italian_food_best_ranked}")
print(f"Nota = {max_rating}")


O restaurante de culinária italiana melhor avaliado é: Darshan
Nota = 4.9


## 5.2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [170]:
colunas = ['restaurant_id','restaurant_name','unique_cuisine','aggregate_rating']
linhas = (df1['unique_cuisine']=='Italian')
df2 = df1.loc[linhas, colunas].copy()
df3 = df2.sort_values(by='aggregate_rating').reset_index()
min_rating = df3.loc[0, 'aggregate_rating']
linhas = df3['aggregate_rating']==min_rating
df4 = df3.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
italian_food_worst_ranked = df4.loc[0,'restaurant_name']
print(f"O restaurante de culinária italiana pior avaliado é: {italian_food_worst_ranked}")
print(f"Nota = {min_rating}")


O restaurante de culinária italiana pior avaliado é: Avenida Paulista
Nota = 0.0


## 5.3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [174]:
colunas = ['restaurant_id','restaurant_name','unique_cuisine','aggregate_rating']
linhas = df1['unique_cuisine']=='American'
df2 = df1.loc[linhas, colunas].sort_values(by='aggregate_rating', ascending=False).reset_index()
max_rating = df2.loc[0,'aggregate_rating']
linhas = df2['aggregate_rating']==max_rating
df3 = df2.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
american_food_best_ranked = df3.loc[0,'restaurant_name']
print(f"O restaurante de culinária americana melhor avaliado é: {american_food_best_ranked}")
print(f"Nota = {max_rating}")

O restaurante de culinária americana melhor avaliado é: Burger & Lobster
Nota = 4.9


## 5.4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [177]:
colunas = ['restaurant_id','restaurant_name','unique_cuisine','aggregate_rating']
linhas = df1['unique_cuisine']=='American'
df2 = df1.loc[linhas, colunas].sort_values(by='aggregate_rating').reset_index()
min_rating = df2.loc[0,'aggregate_rating']
linhas = df2['aggregate_rating']==min_rating
df3 = df2.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
american_food_worst_ranked = df3.loc[0,'restaurant_name']
print(f"O restaurante de culinária americana pior avaliado é: {american_food_worst_ranked}")
print(f"Nota = {min_rating}")

O restaurante de culinária americana pior avaliado é: Alston Bar & Beef
Nota = 0.0


## 5.5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [181]:
colunas = ['restaurant_id','restaurant_name','unique_cuisine','aggregate_rating']
linhas = df1['unique_cuisine']=='Arabian'
df2 = df1.loc[linhas, colunas].sort_values(by='aggregate_rating', ascending=False).reset_index()
max_rating = df2.loc[0,'aggregate_rating']
linhas = df2['aggregate_rating']==max_rating
df3 = df2.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
arabian_food_best_ranked = df3.loc[0,'restaurant_name']
print(f"O restaurante de culinária americana melhor avaliado é: {arabian_food_best_ranked}")
print(f"Nota = {max_rating}")

O restaurante de culinária americana melhor avaliado é: Mandi@36
Nota = 4.7


## 5.6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [183]:
colunas = ['restaurant_id','restaurant_name','unique_cuisine','aggregate_rating']
linhas = df1['unique_cuisine']=='Arabian'
df2 = df1.loc[linhas, colunas].sort_values(by='aggregate_rating').reset_index()
min_rating = df2.loc[0,'aggregate_rating']
linhas = df2['aggregate_rating']==min_rating
df3 = df2.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
arabian_food_best_ranked = df3.loc[0,'restaurant_name']
print(f"O restaurante de culinária americana pior avaliado é: {arabian_food_best_ranked}")
print(f"Nota = {min_rating}")

O restaurante de culinária americana pior avaliado é: Raful
Nota = 0.0


## 5.7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [185]:
colunas = ['restaurant_id','restaurant_name','unique_cuisine','aggregate_rating']
linhas = df1['unique_cuisine']=='Japanese'
df2 = df1.loc[linhas, colunas].sort_values(by='aggregate_rating', ascending=False).reset_index()
max_rating = df2.loc[0,'aggregate_rating']
linhas = df2['aggregate_rating']==max_rating
df3 = df2.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
japanese_food_best_ranked = df3.loc[0,'restaurant_name']
print(f"O restaurante de culinária japonesa melhor avaliado é: {japanese_food_best_ranked}")
print(f"Nota = {max_rating}")

O restaurante de culinária japonesa melhor avaliado é: Sushi Samba
Nota = 4.9


## 5.8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [186]:
colunas = ['restaurant_id','restaurant_name','unique_cuisine','aggregate_rating']
linhas = df1['unique_cuisine']=='Japanese'
df2 = df1.loc[linhas, colunas].sort_values(by='aggregate_rating').reset_index()
min_rating = df2.loc[0,'aggregate_rating']
linhas = df2['aggregate_rating']==min_rating
df3 = df2.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
japanese_food_worst_ranked = df3.loc[0,'restaurant_name']
print(f"O restaurante de culinária japonesa pior avaliado é: {japanese_food_worst_ranked}")
print(f"Nota = {min_rating}")

O restaurante de culinária japonesa pior avaliado é: Banzai Sushi
Nota = 0.0


## 5.9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [189]:
colunas = ['restaurant_id','restaurant_name','unique_cuisine','aggregate_rating']
linhas = df1['unique_cuisine']=='Home-made'
df2 = df1.loc[linhas, colunas].sort_values(by='aggregate_rating', ascending=False).reset_index()
max_rating = df2.loc[0,'aggregate_rating']
linhas = df2['aggregate_rating']==max_rating
df3 = df2.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
homemade_food_best_ranked = df3.loc[0,'restaurant_name']
print(f"O restaurante de culinária caseira melhor avaliado é: {homemade_food_best_ranked}")
print(f"Nota = {max_rating}")

O restaurante de culinária caseira melhor avaliado é: Kanaat Lokantası
Nota = 4.0


## 5.10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [192]:
colunas = ['restaurant_id','restaurant_name','unique_cuisine','aggregate_rating']
linhas = df1['unique_cuisine']=='Home-made'
df2 = df1.loc[linhas, colunas].sort_values(by='aggregate_rating').reset_index()
min_rating = df2.loc[0,'aggregate_rating']
linhas = df2['aggregate_rating']==min_rating
df3 = df2.loc[linhas,:].sort_values(by='restaurant_id').reset_index()
homemade_food_worst_ranked = df3.loc[0,'restaurant_name']
print(f"O restaurante de culinária caseira pior avaliado é: {homemade_food_worst_ranked}")
print(f"Nota = {min_rating}")

O restaurante de culinária caseira pior avaliado é: GurMekan Restaurant
Nota = 3.7


## 5.11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [197]:
colunas = ['unique_cuisine','average_cost_for_two']
df2 = df1.loc[:, colunas].groupby('unique_cuisine').mean().sort_values(by='average_cost_for_two', ascending=False).reset_index()
cuisine_more_expensive_for2 = df2.loc[0, 'unique_cuisine']
print(f"O tipo de culinária com maior preço para duas pessoas é: {cuisine_more_expensive_for2}")

O tipo de culinária com maior preço para duas pessoas é: Modern Australian


## 5.12. Qual o tipo de culinária que possui a maior nota média?

In [199]:
colunas = ['unique_cuisine','aggregate_rating']
df2 = df1.loc[:, colunas].groupby('unique_cuisine').mean().sort_values(by='aggregate_rating', ascending=False).reset_index()
cuisine_best_ranked = df2.loc[0, 'unique_cuisine']
print(f"O tipo de culinária com maior preço para duas pessoas é: {cuisine_best_ranked}")

O tipo de culinária com maior preço para duas pessoas é: Others


,unique_cuisine,aggregate_rating
0,Others,4.900000
1,Ramen,4.800000
2,Egyptian,4.800000
3,Ottoman,4.800000
4,Fresh Fish,4.750000
...,...,...
161,Bengali,2.133333
162,Afghan,2.050000
163,,1.807692
164,Drinks Only,0.000000


## 5.13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [203]:
#   Has Table booking   : int, values = [ 0, 1 ]
#   Has Online delivery : int, values = [ 0, 1 ]
#   Is delivering now   : int, values = [ 0, 1 ]
colunas = ['unique_cuisine','restaurant_id']
linhas = (df1['has_online_delivery']==1) & (df1['is_delivering_now']==1)
df2 = df1.loc[linhas, colunas].groupby('unique_cuisine').count().sort_values(by='restaurant_id', ascending=False).reset_index()
cuisine_more_rest_with_acept_delivery_and_deliver = df2.loc[0, 'unique_cuisine']
print(f"O tipo de culinária com mais restaurantes que aceitam pedidos online e fazem entregas é: {cuisine_more_rest_with_acept_delivery_and_deliver}")

O tipo de culinária com mais restaurantes que aceitam pedidos online e fazem entregas é: North Indian


## 5.14. Quais os TOP 10 restaurantes?

In [27]:
colunas = ['restaurant_id','restaurant_name','country_name','city','unique_cuisine','aggregate_rating']

df3 = df1.loc[:,colunas].copy()
df3['restaurant_id'] = -df3['restaurant_id']
df4 = ( df3.loc[:, :]
           .sort_values(by=['aggregate_rating','restaurant_id'], ascending=False)
           .reset_index()
      )
df4['restaurant_id'] = -df4['restaurant_id']
df4.head(20)


,index,restaurant_id,restaurant_name,country_name,city,unique_cuisine,aggregate_rating
0,3766,7528,Indian Grill Room,India,Gurgaon,North Indian,4.9
1,3758,8913,Pirates of Grill,India,Gurgaon,North Indian,4.9
2,5408,10021,Darshan,India,Pune,Italian,4.9
3,4362,20842,Barbeque Nation,India,Kolkata,North Indian,4.9
4,3018,56618,AB's - Absolute Barbecues,India,Bangalore,European,4.9
5,3038,58882,Byg Brewski Brewing Company,India,Bangalore,Continental,4.9
6,3369,70856,AB's - Absolute Barbecues,India,Chennai,BBQ,4.9
7,7224,75989,Restaurant Mosaic @ The Orient,South Africa,Pretoria,French,4.9
8,4037,101212,Tapri Central,India,Jaipur,Cafe,4.9
9,2683,111826,Yanki Sizzlers,India,Ahmedabad,Continental,4.9


## 5.15. Lista todos os tipos de culinárias

In [8]:
linhas = df1['unique_cuisine'] != ''
df2 = df1.loc[linhas, :].copy()

all_items = df2['unique_cuisine'].unique().tolist()
all_items.sort()
all_items

['Afghan',
 'African',
 'American',
 'Andhra',
 'Arabian',
 'Argentine',
 'Armenian',
 'Asian',
 'Asian Fusion',
 'Assamese',
 'Australian',
 'Author',
 'Awadhi',
 'BBQ',
 'Bakery',
 'Balti',
 'Bar Food',
 'Belgian',
 'Bengali',
 'Beverages',
 'Biryani',
 'Brazilian',
 'Breakfast',
 'British',
 'Burger',
 'Burmese',
 'Cafe',
 'Cafe Food',
 'Cajun',
 'California',
 'Canadian',
 'Cantonese',
 'Caribbean',
 'Charcoal Chicken',
 'Chettinad',
 'Chinese',
 'Coffee',
 'Coffee and Tea',
 'Contemporary',
 'Continental',
 'Creole',
 'Crepes',
 'Cuban',
 'Deli',
 'Desserts',
 'Dim Sum',
 'Dimsum',
 'Diner',
 'Donuts',
 'Drinks Only',
 'Durban',
 'Döner',
 'Eastern European',
 'Egyptian',
 'European',
 'Fast Food',
 'Filipino',
 'Finger Food',
 'Fish and Chips',
 'French',
 'Fresh Fish',
 'Fusion',
 'German',
 'Giblets',
 'Goan',
 'Gourmet Fast Food',
 'Greek',
 'Grill',
 'Gujarati',
 'Hawaiian',
 'Healthy Food',
 'Home-made',
 'Hyderabadi',
 'Ice Cream',
 'Indian',
 'Indonesian',
 'International'